In [2]:
# R.O. Gomes, last update 24.04.2020

%matplotlib inline
import numpy as np
import pandas as pd
import os
from pprint import pprint 
import matplotlib.pyplot as plt
import subprocess

import pandas_profiling


pd.options.display.max_rows = 20

In [3]:
# read data
df_cyclones = pd.read_csv("datasets/OUTPUT_WBI_exposer_cyclones_v3.csv", sep=";")

In [4]:
# features
df_cyclones.columns

Index(['SID', 'NAME', 'ISO', 'YEAR', 'BASIN', 'SUB BASIN', 'NATURE',
       'ISO_TIME', 'COORDS', 'TOTAL_HRS', 'DAY_HRS', 'NIGHT_HRS', 'USA_SSHS',
       'WIND_CALC_MEAN', 'PRES_CALC_MEAN', 'STORM_SPD_MEAN', 'STORM_DR_MEAN',
       'V_LAND_KN', '34KN_ASSETS', '64KN_ASSETS', '96KN_ASSETS', 'CPI',
       'TOTAL_DAMAGE_(000$)', 'TOTAL_DEATHS',
       'Air transport, freight (million ton-km)',
       'Arable land (hectares per person)', 'Cereal yield (kg per hectare)',
       'Food production index (2004-2006 = 100)', 'GDP growth (annual %)',
       'GDP per capita (constant 2010 US$)', 'Net flows from UN agencies US$',
       'Life expectancy at birth, total (years)',
       'Mobile cellular subscriptions (per 100 people)',
       'Population density (people per sq. km of land area)',
       'Adjusted savings: education expenditure (% of GNI)',
       'Rural population (% of total population)', 'Income_level_Final',
       'TOTAL_AFFECTED', 'pop_max_34_adj', 'pop_max_50_adj', 'pop_max_64_

In [8]:
# choosing some numeric features - 
cols = ['YEAR', 'TOTAL_HRS', 'NIGHT_HRS', 'USA_SSHS', 'WIND_CALC_MEAN', 'PRES_CALC_MEAN', 'STORM_SPD_MEAN', 'STORM_DR_MEAN', 
        'V_LAND_KN', 'CPI', 'TOTAL_DAMAGE_(000$)', 'TOTAL_DEATHS', 'Air transport, freight (million ton-km)', 
        'Arable land (hectares per person)', 'Cereal yield (kg per hectare)', 'Food production index (2004-2006 = 100)', 
        'GDP growth (annual %)', 'GDP per capita (constant 2010 US$)', 'Net flows from UN agencies US$', 
        'Life expectancy at birth, total (years)', 'Mobile cellular subscriptions (per 100 people)', 
        'Population density (people per sq. km of land area)', 'Adjusted savings: education expenditure (% of GNI)', 
        'Rural population (% of total population)', 'pop_max_34_adj', 'pop_max_50_adj', 'pop_max_64_adj', 'TOTAL_AFFECTED']

df_cyclones = df_cyclones[cols]
df_cyclones


,YEAR,TOTAL_HRS,NIGHT_HRS,USA_SSHS,WIND_CALC_MEAN,PRES_CALC_MEAN,STORM_SPD_MEAN,STORM_DR_MEAN,V_LAND_KN,CPI,...,Net flows from UN agencies US$,"Life expectancy at birth, total (years)",Mobile cellular subscriptions (per 100 people),Population density (people per sq. km of land area),Adjusted savings: education expenditure (% of GNI),Rural population (% of total population),pop_max_34_adj,pop_max_50_adj,pop_max_64_adj,TOTAL_AFFECTED
0,1949,0,0,0,39.133333,1000.333333,19.000000,91.444444,99.820,9.327572,...,0.0,67.666098,NaN,258.911917,2.867878,36.728000,1.187145e+07,3.193122e+06,5.339257e+05,194046
1,1950,6,6,-1,76.400000,974.833333,23.000000,37.666667,99.820,9.431937,...,0.0,67.666098,NaN,258.911917,2.867878,36.728000,2.845835e+07,1.061136e+07,2.985054e+06,642117
2,1951,15,15,1,88.500000,974.500000,15.500000,283.500000,132.400,10.175534,...,0.0,64.770000,NaN,152.453647,2.578304,66.231000,1.689243e+06,1.599244e+06,6.550875e+05,20200
3,1951,15,15,2,84.488235,979.549020,3.000000,161.647059,119.200,10.175534,...,0.0,61.105000,NaN,91.065000,2.757732,69.703000,4.752653e+06,2.844443e+06,5.896887e+05,60000
4,1952,3,1,1,83.920000,980.600000,10.800000,291.400000,110.691,10.410354,...,0.0,61.105000,NaN,91.065000,2.757732,69.703000,5.119853e+06,2.678770e+06,1.505678e+06,103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000,2019,9,9,3,77.166667,971.611111,3.000000,166.055556,151.200,100.000000,...,0.0,66.012095,35.712976,43.326343,2.080000,63.839788,2.339825e+05,0.000000e+00,0.000000e+00,1100
1001,2019,3,2,3,74.571429,974.714286,8.857143,190.857143,151.200,100.000000,...,0.0,58.508514,51.655359,35.876238,5.380000,64.832034,1.978779e+06,0.000000e+00,0.000000e+00,1501500
1002,2019,3,3,4,121.500000,937.000000,10.000000,248.500000,118.800,100.000000,...,0.0,63.742834,57.220464,432.007513,4.326607,71.300771,NaN,NaN,NaN,345311
1003,2019,0,0,0,75.444444,971.777778,6.777778,239.000000,119.000,100.000000,...,0.0,58.508514,51.655359,35.876238,5.380000,64.832034,3.494627e+06,5.495747e+05,9.133066e+04,400094


In [9]:
df_cyclones.describe()

,YEAR,TOTAL_HRS,NIGHT_HRS,USA_SSHS,WIND_CALC_MEAN,PRES_CALC_MEAN,STORM_SPD_MEAN,STORM_DR_MEAN,V_LAND_KN,CPI,...,Net flows from UN agencies US$,"Life expectancy at birth, total (years)",Mobile cellular subscriptions (per 100 people),Population density (people per sq. km of land area),Adjusted savings: education expenditure (% of GNI),Rural population (% of total population),pop_max_34_adj,pop_max_50_adj,pop_max_64_adj,TOTAL_AFFECTED
count,1005.000000,1005.000000,1005.000000,1005.000000,1001.000000,1005.000000,1005.000000,1005.000000,1005.000000,1005.000000,...,1.005000e+03,1001.000000,996.000000,1005.000000,995.000000,1005.000000,9.240000e+02,9.240000e+02,9.240000e+02,1.005000e+03
mean,1999.845771,13.187065,11.065672,0.379104,59.718462,975.450806,12.047601,215.481903,90.011543,67.801695,...,2.132205e+07,70.647402,43.296536,324.147607,3.244789,47.065884,2.742823e+07,1.415995e+07,4.419502e+06,5.768923e+05
std,14.450944,7.412493,8.259346,1.747668,25.080155,56.845274,6.586180,96.090666,31.633313,24.379109,...,2.632630e+07,7.252531,47.524739,1016.048534,1.848530,22.016753,4.310503e+07,2.695943e+07,9.755211e+06,1.991082e+06
min,1949.000000,0.000000,-1.000000,-5.000000,18.620690,0.000000,1.565217,2.000000,30.000000,9.327572,...,-7.000000e+04,41.762000,0.000000,1.786314,0.538881,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00
25%,1991.000000,6.000000,3.000000,-1.000000,40.000000,966.400000,7.666667,153.800000,62.400000,53.271157,...,1.028399e+06,67.136150,0.052346,50.188247,1.840000,23.959000,3.013991e+06,8.262290e+05,1.569463e+05,3.000000e+03
50%,2004.000000,15.000000,9.000000,0.000000,55.250000,981.857143,10.666667,260.750000,89.200000,73.881412,...,1.201000e+07,71.063000,27.669708,144.583453,2.800000,53.525000,1.086579e+07,3.859164e+06,8.723850e+05,3.000000e+04
75%,2011.000000,21.000000,21.000000,1.000000,75.705882,993.571429,14.411765,288.250000,114.700000,87.984603,...,3.138000e+07,75.164000,83.671391,301.992208,4.270474,64.917000,3.265979e+07,1.513434e+07,3.576978e+06,2.480580e+05
max,2019.000000,23.000000,23.000000,5.000000,150.000000,1013.000000,49.800000,358.000000,171.800000,100.000000,...,1.837500e+08,84.680488,328.790446,20479.769990,14.010000,92.421000,3.758524e+08,3.189559e+08,8.786364e+07,2.962200e+07


In [13]:
# Profiling Report

profile = pandas_profiling.ProfileReport(df_cyclones, minimal = True)
profile.to_file(output_file="profiling-minimal-cyclones.html")